In [1]:
import cv2

# Load video
cap = cv2.VideoCapture('Videos/Captures/duck.mp4')

# Background subtractor with adjusted parameters for less noise
fgbg = cv2.createBackgroundSubtractorMOG2(history=150, varThreshold=20, detectShadows=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame (50% of original size)
    scale_percent = 50
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    frame_resized = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
    
    # Apply a slight Gaussian blur to reduce noise in the input
    frame_blurred = cv2.GaussianBlur(frame_resized, (5, 5), 0)
    
    # Apply background separation to detect motion
    fgmask = fgbg.apply(frame_blurred)
    
   
    # Convert to black and white (binary) for foreground
    _, foreground_mask = cv2.threshold(fgmask, 125, 255, cv2.THRESH_BINARY)
    
    # Foreground alone: Apply mask to original frame
    foreground = cv2.bitwise_and(frame_resized, frame_resized, mask=foreground_mask)
    
    # Background alone: Invert the mask and apply to original frame
    background_mask = cv2.bitwise_not(foreground_mask)
    background = cv2.bitwise_and(frame_resized, frame_resized, mask=background_mask)

    # Display the background alone 
    cv2.imshow('Background Only', background_mask)
    # Display the foreground alone 
    cv2.imshow('Foreground Only', foreground_mask)
    # Wait for any key press to exit
    key = cv2.waitKey(30)
    if key != -1:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture('Videos/Captures/duck.mp4')

# Background subtractor with more sensitive parameters
fgbg = cv2.createBackgroundSubtractorMOG2()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame (50% of original size)
    scale_percent = 50
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    frame_resized = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
    
    # Apply background separation to detect motion
    fgmask = fgbg.apply(frame_resized)
    
    # Threshold with a lower value to capture more motion
    _, motion_mask = cv2.threshold(fgmask, 20, 255, cv2.THRESH_BINARY)
    
    # Light noise reduction with a small kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    motion_mask_cleaned = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel)
    
    # Find contours, but lower the area threshold to keep more motion
    contours, _ = cv2.findContours(motion_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    
    # Create a blank mask for motion only
    motion_only = np.zeros_like(motion_mask_cleaned)
    
    # Draw contours with a lower area threshold
    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Reduced from 100 to capture smaller motion
            cv2.drawContours(motion_only, [contour], -1, (255), thickness=cv2.FILLED)
    
    # Foreground: Apply motion mask to original frame
    foreground = cv2.bitwise_and(frame_resized, frame_resized, mask=motion_only)
    
    # Background: Invert the motion mask and apply to original frame
    background_mask = cv2.bitwise_not(motion_only)
    background = cv2.bitwise_and(frame_resized, frame_resized, mask=background_mask)


    
    # Display the motion-only result (black and white)
    cv2.imshow('Motion Only (Black and White)', motion_only)
    
    # Display the foreground (color)
    cv2.imshow('Foreground', foreground)
    
    # Display the background (color)
    cv2.imshow('Background', background)


     # Create a copy of the frame to draw rectangles on
    frame_with_rectangles = frame_resized.copy()
    
    # Draw rectangles around significant motion
    for contour in contours:
        if cv2.contourArea(contour) > 200:  # Filter out small noise
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame_with_rectangles, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangle
    
    # Display the frame with rectangles around motion
    cv2.imshow('Motion with Rectangles', frame_with_rectangles)

    
    # Wait for any key press to exit
    key = cv2.waitKey(30)
    if key != -1:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [29]:
import cv2
import numpy as np

# Load video
cap = cv2.VideoCapture('Videos/Captures/indoorplay.mp4')

# Background subtractor with balanced parameters
fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold=10, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame (50% of original size)
    scale_percent = 50
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    frame_resized = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)
    
    # Apply light Gaussian blur to reduce minor noise (e.g., court texture)
    frame_blurred = cv2.GaussianBlur(frame_resized, (5, 5), 0)
    
    # Apply background separation to detect motion
    fgmask = fgbg.apply(frame_blurred)
    
    # Threshold to capture player motion, less sensitive to court
    _, motion_mask = cv2.threshold(fgmask, 25, 100, cv2.THRESH_BINARY)
    
    # Noise reduction with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    motion_mask_cleaned = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel)  # Remove small noise
    motion_mask_cleaned = cv2.morphologyEx(motion_mask_cleaned, cv2.MORPH_CLOSE, kernel)  # Fill player gaps
    
    # Contour filtering to isolate players, ignore court noise
    contours, _ = cv2.findContours(motion_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    motion_only = np.zeros_like(motion_mask_cleaned)
    for contour in contours:
        if cv2.contourArea(contour) > 200:  # Larger threshold to focus on players
            cv2.drawContours(motion_only, [contour], -1, (250), thickness=cv2.FILLED)
    
    # Foreground: Apply motion mask to original frame (players alone)
    foreground = cv2.bitwise_and(frame_resized, frame_resized, mask=motion_only)
    
    # Background: Invert the motion mask and apply to original frame
    background_mask = cv2.bitwise_not(motion_only)
    background = cv2.bitwise_and(frame_resized, frame_resized, mask=background_mask)
    
    # Display the motion-only result (black and white)
    cv2.imshow('Motion Only (Black and White)', motion_only)
    
    # Display the foreground (players in color)
    cv2.imshow('Foreground - Players', foreground)
    
    # Display the background (court without players)
    cv2.imshow('Background', background)

    
        # Create a copy of the frame to draw rectangles on
    frame_with_rectangles = frame_resized.copy()
    
    # Draw rectangles around significant motion
    for contour in contours:
        if cv2.contourArea(contour) > 250:  # Filter out small noise
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame_with_rectangles, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangle
    
    # Display the frame with rectangles around motion
    cv2.imshow('Motion with Rectangles', frame_with_rectangles)
    # Wait for any key press to exit
    key = cv2.waitKey(30)
    if key != -1:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [34]:
import cv2
import numpy as np

def process_video(video_path, motion_threshold=250, diff_threshold=25):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open {video_path}")
        return

    # Background subtractor
    fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=15, detectShadows=True)

    frame_count = 0
    prev_motion_area = 0
    scene_change_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        
        # Resize frame (50% scale)
        scale = 50
        width, height = int(frame.shape[1] * scale / 100), int(frame.shape[0] * scale / 100)
        frame_resized = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)

        # Noise reduction
        frame_blurred = cv2.GaussianBlur(frame_resized, (7, 7), 1.5)
        
        # Background subtraction
        fgmask = fgbg.apply(frame_blurred)
        
        # Thresholding
        _, motion_mask = cv2.threshold(fgmask, 30, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        motion_mask_cleaned = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=2)
        motion_mask_cleaned = cv2.morphologyEx(motion_mask_cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Contour detection
        contours, _ = cv2.findContours(motion_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        motion_area = sum(cv2.contourArea(c) for c in contours if cv2.contourArea(c) > motion_threshold)

        # Scene change detection
        area_diff = abs(motion_area - prev_motion_area)
        frame_with_rect = frame_resized.copy()

        # Draw rectangles on every frame
        for contour in contours:
            if cv2.contourArea(contour) > motion_threshold:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame_with_rect, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Highlight scene changes with text
        if area_diff > diff_threshold * 1000:
            scene_change_count += 1
            cv2.putText(frame_with_rect, f"Scene Change {scene_change_count}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            print(f"Scene change at frame {frame_count}, motion area: {motion_area:.0f}, diff: {area_diff:.0f}")

        # Display the processed frame
        cv2.imshow('Processed Video', frame_with_rect)

        # Exit on key press (e.g., 'q')
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

        prev_motion_area = motion_area

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

def main():
    video_path = "Videos/Captures/indoorplay.mp4"  # Input video
    process_video(video_path, motion_threshold=300, diff_threshold=30)

if __name__ == "__main__":
    main()

Scene change at frame 1, motion area: 229401, diff: 229401
Scene change at frame 2, motion area: 4069, diff: 225332
